In [ ]:
import pandas as pd
import requests
from io import StringIO
from time import sleep
import json

In [ ]:
API_KEY = "R7P4PYLQ7HYXMGA4G4WN"
start = "200001"
end = "202412"

## **월별 데이터**

### **원/달러 환율**
- 종가(15:30) 기준
- 월별 평균 환율

In [ ]:
stat_items1 = [{"stat_code": "731Y006", "period": "M", "item_codes": [{"code1": "0000003", "code2": "0000100", "name": "원/달러환율"}]}]

In [ ]:
merged_df1 = None

for stat in stat_items1:
    stat_code = stat["stat_code"]
    period = stat["period"]
    item_codes = stat["item_codes"]

    for item in item_codes:
        item_code1 = item["code1"]
        item_code2 = item["code2"]
        item_name = item["name"]

        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code1}/{item_code2}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if 'StatisticSearch' in data and 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                df_temp = pd.DataFrame(rows)[["TIME", "DATA_VALUE"]].rename(columns={"DATA_VALUE": item_name})
                df_temp["TIME"] = pd.to_datetime(df_temp["TIME"], format="%Y%m").dt.to_period("M")

                if merged_df1 is None:
                    merged_df1 = df_temp
                else:
                    merged_df1 = pd.merge(merged_df1, df_temp, on="TIME", how="outer")
            else:
                print(f"[경고] 데이터 없음 - {item_name} ({item_code1}) ({item_code2})")

            sleep(0.2)

        except Exception as e:
            print(f"[에러] {item_name} ({item_code1}) {item_code2}")

# 결과 확인
merged_df1 = merged_df1.sort_values("TIME").reset_index(drop=True)
merged_df1.head(5)

,TIME,원/달러환율
0,2000-01,1130.32
1,2000-02,1129.29
2,2000-03,1115.75
3,2000-04,1110.01
4,2000-05,1121.37


### **주요 국제 금리**
- 장기 / 단기
- 한 / 미 / 중

In [ ]:
stat_items2 = [
    {"stat_code": "902Y023", "period": "M", "item_codes": [
        {"code1": "IRLT", "code2": "KOR", "name": "한국 장기 시장 금리"},
        {"code1": "IRLT", "code2": "USA", "name": "미국 장기 시장 금리"},
        {"code1": "IRLT", "code2": "CHN", "name": "중국 장기 시장 금리"}
    ]},
    {"stat_code": "902Y023", "period": "M", "item_codes": [
        {"code1": "IR3TIB", "code2": "KOR", "name": "한국 단기 시장 금리"},
        {"code1": "IR3TIB", "code2": "USA", "name": "미국 단기 시장 금리"},
        {"code1": "IR3TIB", "code2": "CHN", "name": "중국 단기 시장 금리"}
    ]}]

In [ ]:
merged_df2 = None

for stat in stat_items2:
    stat_code = stat["stat_code"]
    period = stat["period"]
    item_codes = stat["item_codes"]

    for item in item_codes:
        item_code1 = item["code1"]
        item_code2 = item["code2"]
        item_name = item["name"]

        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code1}/{item_code2}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if 'StatisticSearch' in data and 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                df_temp = pd.DataFrame(rows)[["TIME", "DATA_VALUE"]].rename(columns={"DATA_VALUE": item_name})
                df_temp["TIME"] = pd.to_datetime(df_temp["TIME"], format="%Y%m").dt.to_period("M")

                if merged_df2 is None:
                    merged_df2 = df_temp
                else:
                    merged_df2 = pd.merge(merged_df2, df_temp, on="TIME", how="outer")
            else:
                print(f"[경고] 데이터 없음 - {item_name} ({item_code1}) ({item_code2})")

            sleep(0.2)

        except Exception as e:
            print(f"[에러] {item_name} ({item_code1}) {item_code2}")

# 결과 확인
merged_df2 = merged_df2.sort_values("TIME").reset_index(drop=True)
merged_df2.head(5)

,TIME,한국 장기 시장 금리,미국 장기 시장 금리,중국 장기 시장 금리,한국 단기 시장 금리,미국 단기 시장 금리,중국 단기 시장 금리
0,2000-01,NaN,6.66,NaN,7.28,5.95,2.72
1,2000-02,NaN,6.52,NaN,7.13,6.01,2.73
2,2000-03,NaN,6.26,NaN,7,6.14,2.64
3,2000-04,NaN,5.99,NaN,7.06,6.28,2.64
4,2000-05,NaN,6.44,NaN,7.14,6.71,2.57


### **그 외 월별 데이터**
- 수출 총액
- 수입 총액
- 소득 교역 조건 지수
- 해외 직접 투자 금액
- 경제 심리 지수
- 기준 금리
- 소비자 물가 지수
- 외환 보유액
- 산업 생산 지수
- 실업률
- 주가 지수
- 국제 상품 가격

In [ ]:
stat_items3 = [
    {"stat_code": "901Y011", "period": "M", "item_codes": [
        {"code": "FIEE", "name": "전체 수출 총액"},
        {"code": "FIEE31", "name": "미국 수출 총액"},
        {"code": "FIEE02", "name": "중국 수출 총액"}
    ]},
    {"stat_code": "901Y012", "period": "M", "item_codes": [
        {"code": "FIEF", "name": "전체 수입 총액"},
        {"code": "FIEF31", "name": "미국 수입 총액"},
        {"code": "FIEF02", "name": "중국 수입 총액"}
    ]},

    {"stat_code": "403Y005", "period": "M", "item_codes": [
        {"code": "B", "name": "소득 교역 조건 지수"}
    ]},

    {"stat_code": "901Y060", "period": "M", "item_codes": [
        {"code": "FF1B", "name": "해외 직접 투자 금액"}
    ]},
    {"stat_code": "513Y001", "period": "M", "item_codes": [
        {"code": "E1000", "name": "경제 심리 지수"}
    ]},
    {"stat_code": "902Y006", "period": "M", "item_codes": [
        {"code": "KR", "name": "한국 기준 금리"},
        {"code": "US", "name": "미국 기준 금리"},
        {"code": "CN", "name": "중국 기준 금리"}
    ]},
    {"stat_code": "902Y008", "period": "M", "item_codes": [
        {"code": "KR", "name": "한국 소비자 물가지수"},
        {"code": "US", "name": "미국 소비자 물가지수"},
        {"code": "CN", "name": "중국 소비자 물가지수"}
    ]},

    {"stat_code": "902Y014", "period": "M", "item_codes": [
        {"code": "KR", "name": "한국 외환 보유액"},
        {"code": "US", "name": "미국 외환 보유액"},
        {"code": "CN", "name": "중국 외환 보유액"}
    ]},

    {"stat_code": "902Y020", "period": "M", "item_codes": [
        {"code": "KOR", "name": "한국 산업 생산 지수"},
        {"code": "USA", "name": "미국 산업 생산 지수"}
    ]},

    {"stat_code": "902Y021", "period": "M", "item_codes": [
        {"code": "KOR", "name": "한국 실업률"},
        {"code": "USA", "name": "미국 실업률"}
    ]},

    {"stat_code": "902Y002", "period": "M", "item_codes": [
        {"code": "KOR", "name": "한국 주가지수"},
        {"code": "USA", "name": "미국 주가지수"},
        {"code": "CHN", "name": "중국 주가지수"}
    ]},

    {"stat_code": "902Y003", "period": "M", "item_codes": [
        {"code": "010101", "name": "WTI 유가"},
        {"code": "010102", "name": "두바이 유가"},
        {"code": "010103", "name": "Brent 유가"},
        {"code": "010201", "name": "천연가스 가격"},
        {"code": "010301", "name": "유연탄 가격"},
        {"code": "040205", "name": "철광석 가격"},
        {"code": "040202", "name": "구리 가격"},
        {"code": "040203", "name": "알루미늄 가격"},
        {"code": "040204", "name": "니켈 가격"},
        {"code": "040201", "name": "아연 가격"},
        {"code": "040101", "name": "금 가격"},
        {"code": "030201", "name": "대두 가격"},
        {"code": "030101", "name": "옥수수 가격"},
        {"code": "030102", "name": "소맥 가격"},
        {"code": "030301", "name": "원당 가격"},
        {"code": "020101", "name": "원면 가격"}
    ]}
]

In [ ]:
merged_df3 = None

for stat in stat_items3:
    stat_code = stat["stat_code"]
    period = stat["period"]
    item_codes = stat["item_codes"]

    for item in item_codes:
        item_code = item["code"]
        item_name = item["name"]

        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if 'StatisticSearch' in data and 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                df_temp = pd.DataFrame(rows)[["TIME", "DATA_VALUE"]].rename(columns={"DATA_VALUE": item_name})
                df_temp["TIME"] = pd.to_datetime(df_temp["TIME"], format="%Y%m").dt.to_period("M")

                if merged_df3 is None:
                    merged_df3 = df_temp
                else:
                    merged_df3 = pd.merge(merged_df3, df_temp, on="TIME", how="outer")
            else:
                print(f"[경고] 데이터 없음 - {item_name} ({item_code})")

            sleep(0.2)

        except Exception as e:
            print(f"[에러] {item_name} ({item_code}): {e}")

# 결과 확인
merged_df3 = merged_df3.sort_values("TIME").reset_index(drop=True)
merged_df3.head(5)

,TIME,전체 수출 총액,미국 수출 총액,중국 수출 총액,전체 수입 총액,미국 수입 총액,중국 수입 총액,소득 교역 조건 지수,해외 직접 투자 금액,경제 심리 지수,...,구리 가격,알루미늄 가격,니켈 가격,아연 가격,금 가격,대두 가격,옥수수 가격,소맥 가격,원당 가격,원면 가격
0,2000-01,12162425,2609788,1235215,12594623,2217025,1008166,37.74,244447,NaN,...,1843.85,1679.85,8315.25,1178.58,284.07,180.38,92.95,93.62,17.69,47.43
1,2000-02,12676042,2554310,1273117,12023652,2241520,834291,37.17,298450,NaN,...,1807.03,1679.43,9583.75,1098.25,299.72,185.76,95.08,98.17,17.08,53.44
2,2000-03,14429703,2946821,1437377,14212011,2607380,1125348,42.37,434322,NaN,...,1739.8,1579.64,10255.4,1113,286.92,190.97,95.17,96.4,18.39,57.3
3,2000-04,13522040,2698030,1366877,13344532,2570158,1032362,39.42,342096,NaN,...,1681.91,1458.75,9746.5,1132.52,279.65,197.1,95.54,94.43,19.41,58.77
4,2000-05,14636755,3167181,1663241,13283356,2634243,1066610,43.7,377879,NaN,...,1785.1,1468.07,10122.91,1154.07,276.74,200.87,95.53,99.11,19.12,60.5


## **분기별 데이터**
- 외국인 투자 금액
- 경상 수지
- 상품 수지
- 경제성장률

In [ ]:
stat_items4 = [
    {"stat_code": "901Y061", "period": "Q", "item_codes": [
        {"code": "FF2B", "name": "외국인 투자 금액"}
    ]},
    {"stat_code": "902Y009", "period": "Q", "item_codes": [
        {"code": "KR", "name": "한국 경상수지"},
        {"code": "US", "name": "미국 경상수지"},
        {"code": "CN", "name": "중국 경상수지"}
    ]},
    {"stat_code": "902Y010", "period": "Q", "item_codes": [
        {"code": "KR", "name": "한국 상품수지"},
        {"code": "US", "name": "미국 상품수지"},
        {"code": "CN", "name": "중국 상품수지"}
    ]},
    {"stat_code": "902Y015", "period": "Q", "item_codes": [
        {"code": "KOR", "name": "한국 경제성장률"},
        {"code": "USA", "name": "미국 경제성장률"},
        {"code": "CHN", "name": "중국 경제성장률"}
    ]}]

In [ ]:
merged_df4 = None
start="2000Q1"
end="2024Q4"

for stat in stat_items4:
    stat_code = stat["stat_code"]
    period = stat["period"]
    item_codes = stat["item_codes"]

    for item in item_codes:
        item_code = item["code"]
        item_name = item["name"]

        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if 'StatisticSearch' in data and 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                df_temp = pd.DataFrame(rows)[["TIME", "DATA_VALUE"]].rename(columns={"DATA_VALUE": item_name})

                # 분기 처리 "2021/Q4" → "2021Q4" → Period
                df_temp["TIME"] = df_temp["TIME"].str.replace("/", "")
                df_temp["TIME"] = pd.PeriodIndex(df_temp["TIME"], freq="Q")


                if merged_df4 is None:
                    merged_df4 = df_temp
                else:
                    merged_df4 = pd.merge(merged_df4, df_temp, on="TIME", how="outer")
            else:
                print(f"[경고] 데이터 없음 - {item_name} ({item_code})")

            sleep(0.2)

        except Exception as e:
            print(f"[에러] {item_name} ({item_code}): {e}")

# 결과 확인
merged_df4 = merged_df4.sort_values("TIME").reset_index(drop=True)
merged_df4.head(5)

,TIME,외국인 투자 금액,한국 경상수지,미국 경상수지,중국 경상수지,한국 상품수지,미국 상품수지,중국 상품수지,한국 경제성장률,미국 경제성장률,중국 경제성장률
0,2000Q1,2267485,-266.9,-84585,NaN,948.7,-98066,NaN,1.995,0.363,NaN
1,2000Q2,2999615,2653.9,-96104,NaN,4835,-106480,NaN,1.541,1.821,NaN
2,2000Q3,4725068,3873.8,-114965,NaN,5280.1,-122980,NaN,2.746,0.102,NaN
3,2000Q4,5273193,3920,-106274,NaN,4328.7,-119257,NaN,-0.332,0.597,NaN
4,2001Q1,1538549,303.9,-96454,NaN,1111.8,-105009,NaN,1.006,-0.328,NaN


### **분기별 데이터 → 월별 데이터 변환 및 보간**
- 보간 방법 옵션
  - **linear	가장 일반적인 직선 보간**
  - time	시간 기준 보간
  - spline	곡선 형태 보간 (정교함 ↑)
  - nearest	가장 가까운 값 복사

In [ ]:
# 1. 분기 → 월초 Timestamp
df4qtom = merged_df4.copy()
df4qtom["TIME"] = df4qtom["TIME"].dt.to_timestamp(how="start")

# 2. 숫자형 변환
value_cols = df4qtom.columns.difference(["TIME"])
df4qtom[value_cols] = df4qtom[value_cols].apply(pd.to_numeric, errors="coerce")

# 3. 시간 인덱스 설정
df4qtom = df4qtom.set_index("TIME").resample("MS").asfreq()

# - Linear 보간할 변수들
linear_cols = ["외국인 투자 금액","한국 경상수지", "미국 경상수지", "중국 경상수지", "한국 상품수지", "미국 상품수지", "중국 상품수지", "한국 경제성장률", "미국 경제성장률", "중국 경제성장률"]

# Linear 보간
for col in linear_cols:
    df4qtom[col] = df4qtom[col].interpolate(method="linear")

# 다시 Period[M]로 변환
df4qtom = df4qtom.reset_index()
df4qtom["TIME"] = df4qtom["TIME"].dt.to_period("M")

### **문자형 및 소수점 정리**

In [ ]:
# 분기 → 월 보간 완료된 df4qtom 기준
int_cols = ["외국인 투자 금액"]
float_cols = df4qtom.columns.difference(["TIME"] + int_cols).tolist()

# 정수형: 외국인 투자금액
df4qtom[int_cols] = df4qtom[int_cols].round().astype("Int64")

# 소수점 3자리 반올림: 나머지
df4qtom[float_cols] = df4qtom[float_cols].round(3)

# 결과 확인
df4qtom.head()

,TIME,외국인 투자 금액,한국 경상수지,미국 경상수지,중국 경상수지,한국 상품수지,미국 상품수지,중국 상품수지,한국 경제성장률,미국 경제성장률,중국 경제성장률
0,2000-01,2267485,-266.900,-84585.000,NaN,948.700,-98066.000,NaN,1.995,0.363,NaN
1,2000-02,2511528,706.700,-88424.667,NaN,2244.133,-100870.667,NaN,1.844,0.849,NaN
2,2000-03,2755572,1680.300,-92264.333,NaN,3539.567,-103675.333,NaN,1.692,1.335,NaN
3,2000-04,2999615,2653.900,-96104.000,NaN,4835.000,-106480.000,NaN,1.541,1.821,NaN
4,2000-05,3574766,3060.533,-102391.000,NaN,4983.367,-111980.000,NaN,1.943,1.248,NaN


## **연도별 데이터**
- GDP
- GDP 디플레이터

In [ ]:
stat_items5 = [
    {"stat_code": "902Y016", "period": "A", "item_codes": [
        {"code": "KOR", "name": "한국 GDP"},
        {"code": "USA", "name": "미국 GDP"},
        {"code": "CHN", "name": "중국 GDP"}
    ]},
    {"stat_code": "200Y101", "period": "A", "item_codes": [
        {"code": "90103", "name": "한국 GDP 디플레이터"}
    ]}]

In [ ]:
merged_df5 = None
start="2000"
end="2024"

for stat in stat_items5:
    stat_code = stat["stat_code"]
    period = stat["period"]
    item_codes = stat["item_codes"]

    for item in item_codes:
        item_code = item["code"]
        item_name = item["name"]

        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if 'StatisticSearch' in data and 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                df_temp = pd.DataFrame(rows)[["TIME", "DATA_VALUE"]].rename(columns={"DATA_VALUE": item_name})
                df_temp["TIME"] = pd.PeriodIndex(df_temp["TIME"], freq="Y")


                if merged_df5 is None:
                    merged_df5 = df_temp
                else:
                    merged_df5 = pd.merge(merged_df5, df_temp, on="TIME", how="outer")
            else:
                print(f"[경고] 데이터 없음 - {item_name} ({item_code})")

            sleep(0.2)

        except Exception as e:
            print(f"[에러] {item_name} ({item_code}): {e}")

# 결과 확인
merged_df5 = merged_df5.sort_values("TIME").reset_index(drop=True)
merged_df5.head(5)

,TIME,한국 GDP,미국 GDP,중국 GDP,한국 GDP 디플레이터
0,2000,597801.7,10250952,1223754.9,67.689
1,2001,567654.8,10581929,1355036.5,70.086
2,2002,649747.3,10929108,1489821.6,72.195
3,2003,728497.2,11456450,1683903.4,74.769
4,2004,822583.2,12217196,1984196.6,77.207


In [ ]:
# 분기 → 월초 Timestamp
df5atom = merged_df5.copy()
df5atom["TIME"] = df5atom["TIME"].dt.to_timestamp(how="start")

# 숫자형 변환
value_cols = df5atom.columns.difference(["TIME"])
df5atom[value_cols] = df5atom[value_cols].apply(pd.to_numeric, errors="coerce")

# 시간 인덱스 설정
df5atom = df5atom.set_index("TIME").resample("MS").asfreq()

# Linear 보간할 변수들
linear_cols = ["한국 GDP",	"미국 GDP",	"중국 GDP",	"한국 GDP 디플레이터"]

# Linear 보간
for col in linear_cols:
    df5atom[col] = df5atom[col].interpolate(method="linear")

# 다시 Period[M]로 변환
df5atom = df5atom.reset_index()
df5atom["TIME"]    = pd.PeriodIndex(df5atom["TIME"], freq="M")

## **외부데이터 (중앙정부 부채 비율 - GDP 대비)**
- 연도별 데이터 -> 월별 데이터
- 한,미,중 추출

In [ ]:
from google.colab import files
import pandas as pd

# 2. 불러오기
df = pd.read_csv("/content/general_government_debt.csv")

In [ ]:
# 1. 연도만 추출한 열 리스트
year_cols = [col for col in df.columns if col.isdigit()]

# 2. 연도별로 melt (Country Name, 연도 → 값)
df_melted = df.melt(id_vars=["country_name"], value_vars=year_cols,
                    var_name="TIME", value_name="Value")

# 3. TIME을 연도 숫자형으로 변환
df_melted["TIME"] = df_melted["TIME"].astype(int)

# 4. 피벗: 행 = TIME, 열 = Country Name, 값 = Value
df_pivot = df_melted.pivot(index="TIME", columns="country_name", values="Value")

# 결과 일부 확인
df_pivot.head()


country_name,Albania,Australia,Austria,Belarus,Belgium,Bosnia and Herzegovina,Brazil,Bulgaria,Cambodia,Canada,...,Turkmenistan,"Türkiye, Republic of",Ukraine,United Arab Emirates,United Kingdom,United States,Uzbekistan,Venezuela,Vietnam,Yemen
TIME,,,,,,,,,,,,,,,,,,,,,
2000,63.768684,19.510876,65.737618,NaN,109.589041,34.686286,NaN,71.506010,35.234553,80.436609,...,73.748049,51.318469,43.789532,3.092658,37.672310,53.169404,29.478736,27.700467,24.754757,60.840745
2001,60.612782,17.120759,66.354554,NaN,108.216846,35.230081,67.330659,65.807238,34.868812,81.462033,...,45.595340,75.508902,36.690024,2.689688,34.982533,53.146247,44.838459,30.728852,25.420214,60.568578
2002,64.096916,15.018441,66.977797,NaN,105.436680,31.198184,76.095054,52.971020,39.748321,79.604069,...,32.337455,71.505965,33.579562,3.427151,35.432286,55.537858,41.632893,48.176668,27.711371,57.827228
2003,60.244562,13.188562,64.896277,NaN,101.659311,27.635188,71.513640,45.671312,43.099419,75.911797,...,22.628488,65.159548,29.380226,3.331997,36.724552,58.624539,31.412345,56.028999,29.835244,56.823012
2004,57.474271,11.910367,64.848658,9.222745,97.169665,25.486588,68.025064,37.507236,42.703413,71.889400,...,15.268250,57.158170,24.798092,4.079731,39.778075,66.095288,26.877137,41.783026,29.362706,52.068094


In [ ]:
# 관심 국가만 선택
target_countries = ["Korea, Republic of", "United States"]

# 한,미 필터링
merged_df6 = df_pivot[target_countries].copy()

merged_df6.head()

country_name,"Korea, Republic of",United States
TIME,,
2000,16.681990,53.169404
2001,17.229428,53.146247
2002,17.041285,55.537858
2003,19.803190,58.624539
2004,22.421641,66.095288


In [ ]:
# 컬럼명 변경
merged_df6 = merged_df6.rename(columns={
    "Korea, Republic of" : "한국 중앙정부 부채 비율",
    "United States": "미국 중앙정부 부채 비율"
})


# TIME이 인덱스라면 → 컬럼으로 먼저 변환
if merged_df6.index.name == "TIME":
    merged_df6 = merged_df6.reset_index()

# TIME을 Period[M] (연도-월)로 변환 → 월 초로 지정
merged_df6["TIME"] = pd.PeriodIndex(merged_df6["TIME"].astype(str) + "-01", freq="M")
merged_df6.head()

country_name,TIME,한국 중앙정부 부채 비율,미국 중앙정부 부채 비율
0,2000-01,16.681990,53.169404
1,2001-01,17.229428,53.146247
2,2002-01,17.041285,55.537858
3,2003-01,19.803190,58.624539
4,2004-01,22.421641,66.095288


In [ ]:
# 분기 → 월초 Timestamp
df6atom = merged_df6.copy()
df6atom["TIME"] = df6atom["TIME"].dt.to_timestamp(how="start")

# 숫자형 변환
value_cols = df6atom.columns.difference(["TIME"])
df6atom[value_cols] = df6atom[value_cols].apply(pd.to_numeric, errors="coerce")

# 시간 인덱스 설정
df6atom = df6atom.set_index("TIME").resample("MS").asfreq()

# Linear 보간할 변수들
linear_cols = ["한국 중앙정부 부채 비율",	"미국 중앙정부 부채 비율"]

# Linear 보간
for col in linear_cols:
    df6atom[col] = df6atom[col].interpolate(method="linear")

# 다시 Period[M]로 변환
df6atom = df6atom.reset_index()
df6atom["TIME"]    = pd.PeriodIndex(df6atom["TIME"], freq="M")

## **데이터 병합**

In [ ]:
# 데이터 병합
merged_all = pd.merge(merged_df1, merged_df2, on="TIME", how="outer")
merged_all = pd.merge(merged_all, merged_df3, on="TIME", how="outer")
merged_all = pd.merge(merged_all, df4qtom, on="TIME", how="outer")
merged_all = pd.merge(merged_all, df5atom, on="TIME", how="outer")
merged_all = pd.merge(merged_all, df6atom, on="TIME", how="outer")

# 시간 순 정렬
merged_all = merged_all.sort_values("TIME").reset_index(drop=True)
merged_all.head()

,TIME,원/달러환율,한국 장기 시장 금리,미국 장기 시장 금리,중국 장기 시장 금리,한국 단기 시장 금리,미국 단기 시장 금리,중국 단기 시장 금리,전체 수출 총액,미국 수출 총액,...,중국 상품수지,한국 경제성장률,미국 경제성장률,중국 경제성장률,한국 GDP,미국 GDP,중국 GDP,한국 GDP 디플레이터,한국 중앙정부 부채 비율,미국 중앙정부 부채 비율
0,2000-01,1130.32,NaN,6.66,NaN,7.28,5.95,2.72,12162425,2609788,...,NaN,1.995,0.363,NaN,597801.700000,1.025095e+07,1.223755e+06,67.68900,16.681990,53.169404
1,2000-02,1129.29,NaN,6.52,NaN,7.13,6.01,2.73,12676042,2554310,...,NaN,1.844,0.849,NaN,595289.458333,1.027853e+07,1.234695e+06,67.88875,16.727609,53.167474
2,2000-03,1115.75,NaN,6.26,NaN,7,6.14,2.64,14429703,2946821,...,NaN,1.692,1.335,NaN,592777.216667,1.030611e+07,1.245635e+06,68.08850,16.773229,53.165544
3,2000-04,1110.01,NaN,5.99,NaN,7.06,6.28,2.64,13522040,2698030,...,NaN,1.541,1.821,NaN,590264.975000,1.033370e+07,1.256575e+06,68.28825,16.818849,53.163615
4,2000-05,1121.37,NaN,6.44,NaN,7.14,6.71,2.57,14636755,3167181,...,NaN,1.943,1.248,NaN,587752.733333,1.036128e+07,1.267515e+06,68.48800,16.864469,53.161685


In [ ]:
print(merged_all.isnull().sum().to_string())

TIME               0
원/달러환율             0
한국 장기 시장 금리        9
미국 장기 시장 금리        0
중국 장기 시장 금리      168
한국 단기 시장 금리        0
미국 단기 시장 금리        1
중국 단기 시장 금리        1
전체 수출 총액           0
미국 수출 총액           0
중국 수출 총액           0
전체 수입 총액           0
미국 수입 총액           0
중국 수입 총액           0
소득 교역 조건 지수        0
해외 직접 투자 금액        0
경제 심리 지수          36
한국 기준 금리           0
미국 기준 금리           0
중국 기준 금리           0
한국 소비자 물가지수        0
미국 소비자 물가지수        0
중국 소비자 물가지수        0
한국 외환 보유액         60
미국 외환 보유액         20
중국 외환 보유액        185
한국 산업 생산 지수        0
미국 산업 생산 지수        0
한국 실업률             0
미국 실업률             0
한국 주가지수            0
미국 주가지수            0
중국 주가지수            0
WTI 유가             0
두바이 유가             0
Brent 유가           0
천연가스 가격            0
유연탄 가격             0
철광석 가격             0
구리 가격              0
알루미늄 가격            0
니켈 가격              0
아연 가격              0
금 가격               0
대두 가격              0
옥수수 가격             0
소맥 가격              0
원당 가격        

In [ ]:
merged_all.to_csv("/content/exchange_2000_2024.csv", index=False, encoding="utf-8-sig")